# Ch08. 딥 러닝(Deep Learning) 개요

# v06. 케라스(Keras) 훑어보기

- 이 책에서는 딥 러닝을 쉽게 할 수 있는 파이썬 라이브러리인 케라스(Keras)를 사용한다.
- 케라스는 유저가 손쉽게 딥 러닝을 구현할 수 있도록 도와주는 상위 레벨의 인터페이스이다.
- 케라스를 사용하면 딥 러닝을 쉽게 구현할 수 있다.
- [케라스 공식 문서](https://keras.io/)

<br>

## 6.1 전처리 (Preprocessing)

### 6.1.1 `Tokenizer()`

- 토큰화와 정수 인코딩(단어에 대한 인덱싱)을 위해 사용된다.
- [정수 인코딩 챕터](https://github.com/sunghan-kim/Introduction-to-NLP-Using-DL/blob/master/Ch02_Text-preprocessing/Ch02_v06_Integer_Encoding.ipynb) 참고

In [4]:
%tensorflow_version 2.x

import tensorflow as tf
tf.__version__

'2.1.0'

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

t = Tokenizer()

fit_text = "The earth is an awesome place live"

t.fit_on_texts([fit_text])

test_text = "The earth is an great place live"

sequences = t.texts_to_sequences([test_text])[0]

print("sequences : " , sequences) # great는 단어 집합(vocabulary)에 없으므로 출력되지 않는다.
print("word_index : ", t.word_index) # 단어 집합(vocabulary) 출력

sequences :  [1, 2, 3, 4, 6, 7]
word_index :  {'the': 1, 'earth': 2, 'is': 3, 'an': 4, 'awesome': 5, 'place': 6, 'live': 7}


<br>

### 6.1.2 `pad_sequences()`

- 전체 훈련 데이터에서 각 샘플의 길이는 서로 다를 수 있다.
- 또는 각 문서, 각 문장은 단어의 수가 제각각이다.
- 모델의 입력으로 사용하려면 모든 샘플의 길이를 동일하게 맞춰야 할 때가 있다.
- 이를 자연어 처리에서는 **패딩(padding) 작업**이라고 한다.
- 보통 숫자 0을 넣어서 길이가 다른 샘플들의 길이를 맞춰준다.
- 케라스에서는 `pad_sequences()`를 사용한다.
- `pad_sequences()`는 정해준 길이보다 길이가 긴 샘플은 값을 일부 자르고, 정해준 길이보다 길이가 짧은 샘플은 값을 0으로 채운다.

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리가 끝나서 각 단어에 대한 정수 인코딩이 끝났다고 가정하고, 3개의 데이터를 입력으로 한다.
pad_sequences([[1, 2, 3],
               [3, 4, 5, 6],
               [7, 8]],
              maxlen=3,
              padding='pre')

array([[1, 2, 3],
       [4, 5, 6],
       [0, 7, 8]], dtype=int32)

- `첫 번째 인자` : 패딩을 진행할 데이터
- `maxlen` : 모든 데이터에 대해서 정규화할 길이
- `padding`
  - `padding='pre'` : 앞에 0을 채움
  - `padding='post'` : 뒤에 0을 채움

<br>

## 6.2 워드 임베딩 (Word Embedding)

- 워드 임베딩이란 텍스트 내의 단어들을 밀집 벡터(dense vector)로 만드는 것을 말한다.

### 6.2.1 희소 벡터 (sparse vector)

- 원-핫 벡터는 대부분이 0의 값을 가지고, 단 하나의 1의 값을 가지는 벡터이다.
- 또한 벡터의 차원이 대체적으로 크다는 성질을 가진다.
- 원-핫 벡터의 예 : `[0 1 0 0 0 0 0 ... 0 0 0 0 0 0 0]`

- 대부분의 값이 0인 이러한 벡터를 **희소 벡터(sparse vector)**라고 한다.
- 원-핫 벡터는 희소 벡터의 예이다.
- 원-핫 벡터는 단어의 수 만큼 벡터의 차원을 가지며 단어 간 유사도가 모두 동일하다는 단점이 있다.

<br>

### 6.2.2 밀집 벡터 (dense vector)

- 반면, 희소 벡터와 표기상으로도 의미상으로도 반대인 벡터가 있다.
- 대부분의 값이 실수이고, 상대적으로 저차원인 **밀집 벡터(dense vector)**이다.
- 밀집 벡터의 예 : `[0.1 -1.2 0.8 0.2 1.8]`

<br>

### 6.2.3 희소 벡터 vs 밀집 벡터

| -         | 원-핫 벡터               | 임베딩 벡터              |
| :-------- | :----------------------- | :----------------------- |
| 차원      | 고차원(단어 집합의 크기) | 저차원                   |
| 다른 표현 | 희소 벡터의 일종         | 밀집 벡터의 일종         |
| 표현 방법 | 수동                     | 훈련 데이터로부터 학습함 |
| 값의 타입 | 1과 0                    | 실수                     |

<br>

### 6.2.4 단어를 벡터로 만드는 과정

#### 6.2.4.1 원-핫 인코딩

- 단어를 원-핫 벡터로 만드는 과정을 원-핫 인코딩이라고 한다.

<br>

#### 6.2.4.2 워드 임베딩 (word embedding)

- 단어를 밀집 벡터로 만드는 작업을 **워드 임베딩(word embedding)**이라고 한다.
- 밀집 벡터
  - 워드 임베딩 과정을 통해 나온 결과
  - 임베딩 벡터(embedding vector)라고도 함

<br>

### 6.2.5 벡터의 차원

#### 6.2.5.1 원-핫 벡터의 차원

- 원-핫 벡터의 차원은 주로 20,000 이상을 넘어간다.

<br>

#### 6.2.5.2 임베딩 벡터의 차원

- 임베딩 벡터는 주로 256, 512, 1024 등의 차원을 가진다.

<br>

### 6.2.6 임베딩 벡터의 값 결정

- 임베딩 벡터는 초기에는 랜덤값을 가진다.
- 인공 신경망의 가중치가 학습되는 방법과 같은 방식으로 값이 학습되며 변경된다.

<br>

### 6.2.7 `Embedding()`

- `Embedding()`은 단어를 밀집 벡터로 만드는 역할을 한다.
- 인공 신경망 용어로 **임베딩 층(embedding layer)**을 만드는 역할을 한다.
- `Embedding()`은 정수 인코딩이 된 단어들을 입력으로 받아서 임베딩을 수행한다.

<br>

#### 6.2.7.1 `Embedding()`의 입력값

- `Embedding()`은 `(number of samples, input_length)`인 2D 정수 텐서를 입력받는다.
- 이 때 각 `sample`은 정수 인코딩된 결과이다. (정수 시퀀스)

<br>

#### 6.2.7.2 `Embedding()`의 출력값

- `Embedding()`은 워드 임베딩 작업을 수행하고 `(number of samples, input_length, embedding word dimentionality)`인 3D 텐서를 리턴한다.

<br>

#### 6.2.7.3 임베딩 의사 코드(pseudo-code)

```python
# 문장 토큰화와 단어 토큰화
text = [['Hope', 'to', 'see', 'you', 'soon'],
        ['Nice', 'to', 'see', 'you', 'again']]

# 각 단어에 대한 정수 인코딩
text = [[0, 1, 2, 3, 4],
        [5, 1, 2, 3, 6]]

# 위 데이터가 아래의 임베딩 층의 입력이 된다.
Embedding(7, 2, input_length=5)
# 7 : 단어의 개수 (즉, 단어 집합(vocabulary)의 크기)
# 2 : 임베딩한 후의 벡터의 크기
# 5 : 각 입력 시퀀스의 길이 (즉, input_length)

# 각 정수는 아래의 테이블의 인덱스로 사용되며 Embedding()은 각 단어에 대해 임베딩 벡터를 리턴한다.
+------------+------------+
|   index    | embedding  |
+------------+------------+
|     0      | [1.2, 3.1] |
|     1      | [0.1, 4.2] |
|     2      | [1.0, 3.1] |
|     3      | [0.3, 2.1] |
|     4      | [2.2, 1.4] |
|     5      | [0.7, 1.7] |
|     6      | [4.1, 2.0] |
+------------+------------+
# 위의 표는 임베딩 벡터가 된 결과를 예로서 정리한 것이고 Embedding()의 출력인 3D 텐서를 보여주는 것이 아님.
```

<br>

#### 6.2.7.4 `Embedding()`의 인자

- `첫 번째 인자` : 단어 집합의 크기 (즉, 총 단어의 개수)
- `두 번째 인자` : 임베딩 벡터의 출력 차원 (결과로서 나오는 임베딩 벡터의 크기)
- `input_length` : 입력 시퀀스의 길이

<br>

## 6.3 모델링 (Modeling)

### 6.3.1 `Sequential()`

- 케라스에서는 입력층, 은닉층, 출력층과 같은 층을 구성하기 위해 `Sequential()`을 사용한다.
- `Sequential()`을 `model`로 선언한 뒤에 `model.add()`라는 코드를 통해 층을 단계적으로 추가한다.
```python
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(...)
model.add(...)
model.add(...)
```

- `Embedding()`을 통해 생성하는 임베딩 층(embedding layer) 또한 인공 신경망의 층의 하나이므로 `model.add()`로 추가해야 한다.
```python
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(vocabulary, output_dim, input_length))
```

<br>

### 6.3.2 `Dense()`

- 전결합층(fully-connected layer)을 추가한다.
- `model.add()`를 통해 추가할 수 있다.
```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(1, input_dim=3, activation='relu'))
```

<br>

#### 6.3.2.1 `Dense()`의 대표적인 인자

- `첫 번째 인자` : 출력 뉴런의 수
- `input_dim` : 입력 뉴런의 수 (입력의 차원)
- `activation` : 활성화 함수
  - `linear` : 디폴트 값, 별도의 활성화 함수 없이 입력 뉴런과 가중치의 계산 결과를 그대로 출력 (ex. 선형 회귀)
  - `sigmoid` : 시그모이드 함수, **이진 분류 문제**에서 **출력층**에 주로 사용하는 활성화 함수
  - `softmax` : 소프트맥스 함수, 셋 이상을 분류하는 **다중 클래스 분류 문제**에서 **출력층**에 주로 사용되는 활성화 함수
  - `relu` : 렐루 함수, **은닉층**에 주로 사용되는 활성화 함수

<br>

#### 6.3.2.2 `Dense()`의 의미

- 첫 번째 인자값 = 1 $\rightarrow$ 총 1개의 출력 뉴런을 의미한다.
- 두 번째 인자 `input_dim` : 입력층의 뉴런의 수 의미 (위 경우에는 3)
- 이를 통해 3개의 입력층 뉴런과 1개의 출력층 뉴런을 만들었다.
- 이를 시각화하면 다음과 같다.

$\quad$ ![](https://wikidocs.net/images/page/32105/neural_network1_final.PNG)

<br>

#### 6.3.2.3 2개의 전결합층 사용

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(8, input_dim=4, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # 출력층
```

- 이번에는 `Dense()`가 두 번 사용되었다.
- `Dense()`가 처음 사용되었을 때와 추가로 사용되었을 때의 인자는 조금 다르다.
- 이제 첫 번째 사용된 `Dense()`의 8이라는 값은 더 이상 출력층의 뉴런이 아니라 은닉층의 뉴런이다.
- 뒤에 층이 하나 더 생겼기 때문이다.

- 두 번째 `Dense()`는 `input_dim` 인자가 없다.
- 이는 이미 이전층의 뉴런의 수가 8개라는 사실을 알고 있기 때문이다.
- 위의 코드에서 두 번째 `Dense()`는 마지막 층이므로, 첫 번째 인자 1은 결국 출력층의 뉴런의 개수가 된다.
- 이를 시각화하면 다음과 같다.

$\quad$ ![](https://wikidocs.net/images/page/32105/neural_network2_final.PNG)

<br>

#### 6.3.2.4 `Dense()` 이외의 다양한 층들

- `LSTM`
- `GRU`
- `Convolution2D`
- `BatchNormalization`

<br>

### 6.3.3 `summary()`

- 모델의 정보를 요약해서 보여준다.

```python
model.summary()
```

```python
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_1 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
=================================================================
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________
```

<br>

## 6.4 컴파일(Compile)과 훈련(Training)

### 6.4.1 `compile()`

- 모델을 기계가 이해할 수 있도록 컴파일한다.
- 오차 함수와 최적화 방법, 메트릭 함수를 선택할 수 있다.

```python
# 이 코드는 뒤의 텍스트 분류 챕터의 스팸 메일 분류하기 실습 코드를 갖고온 것임
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

max_features = 10000

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
```

- 위의 코드는 임베딩 층, 은닉층, 출력층을 추가하여 모델을 설계한 후에 마지막으로 컴파일하는 과정을 보여준다.

<br>

#### 6.4.1.1 `compile()`의 주요 인자

- `optimizer` : 훈련 과정에서 설정하는 옵티마이저를 설정, `'adam'`, `'sgd'`와 같이 문자열로 지정할 수도 있다.
- `loss` : 훈련 과정에서 사용할 손실 함수(loss function)를 설정
- `metrics` : 훈련을 모니터링하기 위한 지표를 선택

<br>

### 6.4.2 대표적을로 사용되는 손실 함수와 활성화 함수의 조합

- 더 많은 함수는 케라스 공식 문서에서 확인 가능하다.

| 문제유형         | 손실 함수                                              | 출력층의 활성화 함수 | 참고 설명                                                    |
| ---------------- | ------------------------------------------------------ | -------------------- | ------------------------------------------------------------ |
| 회귀 문제        | `mean_squared_error`<br />(평균 제곱 오차)             | -                    | -                                                            |
| 다중 클래스 분류 | `categorical_crossentropy`<br />(범주형 교차 엔트로피) | 소프트맥스           | Ch10 로이터 뉴스 분류하기 실습 참고                          |
| 다중 클래스 분류 | `sparse_categorical_crossentropy`                      | 소프트맥스           | 범주형 교차 크로스엔트로피와 동일<br />하지만 이 경우 원-핫 인코딩이 된 상태일 필요 없이 정수 인코딩된 상태에서 수행 가능 |
| 이진 분류        | `binary_crossentropy`<br />(이항 교차 엔트로피)        | 시그모이드           | Ch10 스팸 메일 분류하기, IMDB 리뷰 감성 분류하기 실습 참고   |



<br>

### 6.4.3 `fit()`

- 진행중...